In [1]:
from lotc.trainer import FileBinaryClassifierTrainer

In [ ]:
pos_data = './data/pos'
neg_data = './data/neg'
trainer = FileBinaryClassifierTrainer(
    model_name='logistic_regression_example',
    max_length=4096,
    num_epochs=10,
    train_loss_threshold=15
)
model = trainer.train(pos_data, neg_data)
trainer.save(model)

In [5]:
from lotc.embedding import long_text_embedding
model = trainer.load(model)
model.predict(long_text_embedding('这是一个测试文本', max_length=4000, chunk_size=256))

AttributeError: 'FileBinaryClassifierTrainer' object has no attribute 'load'